Here I am using Haleigh Miller's platemappy code but adjusted it to my BWF cohort. 

Authors: ChatGPT and Sophie Porak

In [ ]:
import pandas as pd 
import random 
import numpy as np 
import math 
import matplotlib.pyplot as plt 
from matplotlib.colors import BoundaryNorm, ListedColormap
import matplotlib.lines as mlines

def chunks(lst, n):
    """
    Split list 'lst' into n (as even as possible) chunks. 
    Returns a list of lists with length n. Some may differ by max 1 element.

    This function ensures that data gets split into chunks/groups as evenly as possible! 
    """
    n = max(1, n)
    q, r = divmod(len(lst), n) #returns quotient and remainder e.g. divmod(5, 2) = (2, 1), as 2 x 2 + 1 = 5
    out = []
    start = 0
    for i in range(n):
        size = q + (1 if i < r else 0)
        out.append(lst[start:start+size])
        start += size 
    return out


class Metadata:
    """
    Plate map generator that support any number of sample groups.

    Parameters:
    ----------
    X : pd.DataFrame. 
        This is the metadata table. Index must be unique sample IDs (strings).

    group_col : str
        Column that holds the category label for each sample.
        Examples: 'History of BWF' 'Incident F1' 'control' etc.

    n_ag : int
        Number of AG controls per plate

    n_canary : int 
        Number of Canary control wells per plate

    n_mAb : int 
        Count of mAb controls per plate.

    duplicate_ : int 
        Number of technical duplicates per sample (e.g. 2 means duplicates for each sample)

    block_ : int, default 0
        Size of a contriguous block of wells to insert (filled with 'HC')
    
    block_column_ : list[int], default []
        Zero-based column indices to block entirely (0..11). Blockers filled with 0. 
    
    block_row_ : list[int], default []
        Zero-based row indices to block entirely (0..7). Blockers filled with 0. 

    Attributes
    -----------
    k : int
        Number of plates
    
    cats : list[str]
        Ordered list of unique group labels found in X[group_col]

    """
    
    def __init__(
            self, 
            X : pd.DataFrame, 
            group_col: str, 
            n_ag: int, 
            n_canary: int, 
            n_mAb: int, 
            duplicate_: int, 
            block_ = 0, 
            block_column_ = None, 
            block_row_ = None, 
    ) -> None: 
        if block_column_ is None: block_column_ = []
        if block_row_ is None: block_row_ = []

        assert X.index.is_unique, "X.index (sample IDs) must be unique."
        assert group_col in X.columns, f"{group_col} not in X.columns"

        self._X = X.copy()
        self._group = group_col 
        self._ag = int(n_ag)
        self._can = int(n_canary)
        self._mab = int(n_mAb)
        self._block = int(block_)
        self._dup = int(duplicate_)
        self._blockcol = list(block_column_)
        self._blockrow = list(block_row_)

        #categories (groups) present in metadata 
        self.cats = list(self._X[self._group].astype(str).unique())

        self.k = self._init_params()

    def _init_params(self) -> int:
        """
        Compute the number of plates given controls + blocks.
        """
        x = (
            self._ag + self._can + self._mab
            + self._block 
            + 12 * len(self._blockcol)
            + 12 * len(self._blockrow)
        )

        usable = max(0, 96 - x)
        total_samples = self._X.shape[0] * (self._dup if self._dup else 1)
        k = math.ceil(total_samples / usable) if usable else math.ceil(total_samples / 1) #math.ceil rounds number up to nearest integer. e.g. math.ceil(4.2) = 5. in this case ensures there are enough slots / wells.
        return max(1, k)
    

